In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load The Data

In [ ]:
train = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().sum()

In [ ]:
shape_before= train.shape

# (Exploratory Data Analysis)
## Analysis and Reducing ["Category"] Column

In [ ]:
train.drop_duplicates(subset= None, keep="first", inplace=True)


shape_after= train.shape


print('org_samples:', shape_before,
      '\ndrop_sample:', shape_after,
      '\nNo. Dropping Sample:', shape_before[0], '-', shape_after[0], '=', shape_before[0] - shape_after[0],
     'Samples') 

In [ ]:
pd.DataFrame(train['Category'].value_counts(ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, count = plt.subplots(figsize = (17,15))
count = sns.countplot(y = train['Category'])
count.set_title('Crime counts')

In [ ]:
category_freq = pd.DataFrame(train['Category'].value_counts(normalize=True))

category_freq.reset_index(inplace=True)

category_freq.rename({'index': 'Category', 'Category': 'Frequency'}, axis='columns', inplace=True)

category_freq['Cumulative_Frequency'] = category_freq['Frequency'].cumsum()

category_freq

In [ ]:
candidate_classes = list(category_freq.loc[category_freq['Cumulative_Frequency'] < 0.8, 'Category'])
candidate_classes

In [ ]:
train_reduced_categories = train.loc[train['Category'].isin(candidate_classes)]
train_reduced_categories 

In [ ]:
fig, count = plt.subplots(figsize = (15,5))
count = sns.countplot(y = train_reduced_categories['Category'])
count.set_title('Most common types of crimes')

In [ ]:
warrants = train_reduced_categories.loc[train_reduced_categories['Category'] == 'WARRANTS']
warrants

In [ ]:
warrants['Descript'].value_counts()

In [ ]:
fig, count = plt.subplots(figsize = (15,5))
count = sns.countplot(y = warrants['Descript'])
count.set_title('Warrant Descriptions Counts')

In [ ]:
others = train_reduced_categories.loc[train_reduced_categories['Category'] == 'OTHER OFFENSES']
others

In [ ]:
others['Descript'].unique()

In [ ]:
larceny_theft = train_reduced_categories.loc[train_reduced_categories['Category'] == 'LARCENY/THEFT']
larceny_theft

In [ ]:
larceny_theft['Descript'].unique()

In [ ]:
vehicle_theft = train_reduced_categories.loc[train_reduced_categories['Category'] == 'VEHICLE THEFT']
vehicle_theft

In [ ]:
vehicle_theft['Descript'].value_counts()

In [ ]:
fig, count = plt.subplots(figsize = (15,10))
count = sns.countplot(y = vehicle_theft['Descript'])
count.set_title('Vehicle Theft Descriptions Count')

In [ ]:
vandalism = train_reduced_categories.loc[train_reduced_categories['Category'] == 'VANDALISM']
vandalism

In [ ]:
vandalism['Descript'].value_counts()

In [ ]:
fig, count = plt.subplots(figsize = (15,10))
count = sns.countplot(y = vandalism['Descript'])
count.set_title('Vandalism Descriptions Count')

In [ ]:
non_criminal = train_reduced_categories.loc[train_reduced_categories['Category'] == 'NON-CRIMINAL']
non_criminal

In [ ]:
non_criminal['Descript'].value_counts()

In [ ]:
fig, count = plt.subplots(figsize = (17,15))
count = sns.countplot(y = non_criminal['Descript'])
count.set_title('Non-Criminal Events Descriptions Count')

In [ ]:
assault = train_reduced_categories.loc[train_reduced_categories['Category'] == 'ASSAULT']
assault

In [ ]:
assault['Descript'].unique()

In [ ]:
train_reduced_categories.loc[train_reduced_categories['Category'].isin(['LARCENY/THEFT', 'VEHICLE THEFT']), 'Category'] = 'THEFT'
train_reduced_categories['Category'].unique()

In [ ]:
fig, count = plt.subplots(figsize = (15,5))
count = sns.countplot(y = train_reduced_categories['Category'])
count.set_title('Most common types of crimes')

In [ ]:
train_clean = train_reduced_categories

# Analysis ["PdDistrict"] Column 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, count = plt.subplots(figsize = (12,10))
count = sns.countplot(x = train_clean['PdDistrict'])
count.set_title('Districts Count')

In [ ]:
ct_district_cat = pd.crosstab(train_clean['Category'], train_clean['PdDistrict'])
ct_district_cat

# npp = np.array(ct_district_cat)
# sum = 0
# for i in range(0, npp.shape[0]):
#     for j in range(0, npp.shape[1]):
#         sum+= npp[i][j]
# sum

In [ ]:
stacked= ct_district_cat.stack().reset_index().rename(columns= {0:'value'})
fig, bar= plt.subplots(figsize= (15,10))
bar= sns.barplot(x= stacked['PdDistrict'], y= stacked['value'], hue= stacked['Category'])
bar.set_title('Categories Count per District')

In [ ]:
fig, heatmap = plt.subplots(figsize = (15,10))
heatmap = sns.heatmap(ct_district_cat, annot= True, cmap= 'Reds')
heatmap.set_title('Categories vs Districts')

## Features Engineering

In [ ]:
sorted_days = {'Saturday': 0, 'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6}

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'ASSAULT'].groupby(by=['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15,5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x = data['DayOfWeek'], y=data['Category'])
line.set_title('Assault Crimes over the week')

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'BURGLARY'].groupby(by= ['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15, 5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x= data['DayOfWeek'], y= data['Category'])
line.set_title('Burglary Crimes over the week')

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'BURGLARY'].groupby(by= ['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15, 5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x= data['DayOfWeek'], y= data['Category'])
line.set_title('Burglary Crimes over the week')

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'THEFT'].groupby(by=['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15,5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x = data['DayOfWeek'], y=data['Category'])
line.set_title('Theft Crimes over the week')

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'VANDALISM'].groupby(by=['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15,5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x = data['DayOfWeek'], y=data['Category'])
line.set_title('Vandalism Crimes over the week')

In [ ]:
data = pd.DataFrame(train_clean[train_clean['Category'] == 'WARRANTS'].groupby(by=['DayOfWeek'])['Category'].count()).reset_index()
fig, line = plt.subplots(figsize = (15,5))

data['SortedDayOfWeek'] = data['DayOfWeek'].map(sorted_days)
data = data.sort_values('SortedDayOfWeek').drop('SortedDayOfWeek', axis=1)

line = sns.lineplot(x = data['DayOfWeek'], y=data['Category'])
line.set_title('Warrants issued over the week')

In [ ]:
train_clean["Dates"] = pd.to_datetime(train_clean["Dates"])
train_clean["YEAR"] = train_clean["Dates"].dt.year
train_clean["MONTH"] = train_clean["Dates"].dt.month
train_clean["DAY"] = train_clean["Dates"].dt.day
train_clean["HOUR"] = train_clean["Dates"].dt.hour
train_clean

In [ ]:
data = pd.DataFrame(train_clean.groupby('YEAR')['Category'].count())
data.reset_index(inplace=True)
data

# data['Category'].sum()

In [ ]:
train_clean['Dates'].max()

In [ ]:
data.drop(12, axis='rows', inplace=True)
data

In [ ]:
fig, line = plt.subplots(figsize = (15,5))
line = sns.lineplot(x = data['YEAR'], y=data['Category'])
line.set_title('Crimes recorded over the years')

In [ ]:
ct_year_cat = pd.crosstab(train_clean['Category'], train_clean['YEAR'])
ct_year_cat

# npp = np.array(ct_year_cat)
# sum = 0
# for i in range(0, npp.shape[0]):
#     for j in range(0, npp.shape[1]):
#         sum+= npp[i][j]
# sum

In [ ]:
fig, line = plt.subplots(figsize = (15,5))
line = sns.lineplot(x = data['YEAR'], y=data['Category'])
line.set_title('Crimes recorded over the years')

In [ ]:
stacked = ct_year_cat.stack().reset_index().rename(columns={0:'value'})
filt = stacked['YEAR'] != 2015
stacked = stacked[filt]

fig, count = plt.subplots(figsize = (15,10))
line = sns.lineplot(x= stacked['YEAR'], y= stacked['value'], hue=stacked['Category'])
line.set_title('Different crimes recorded over the years')

In [ ]:
def get_time_periods(hour):
    if hour >= 2 and hour < 8:
        return 'Dawn'
    elif hour >= 8 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 16:
        return 'Afternoon'
    elif hour >= 16 and hour < 20:
        return 'Evening'
    elif hour >= 20 and hour < 23:
        return 'Night'
    else:
        return 'Late Night'

In [ ]:
train_clean['Day Period'] = train_clean['HOUR'].map(get_time_periods)
train_clean['Day Period'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.DataFrame(train_clean['Day Period'].value_counts()).reset_index()
fig, count = plt.subplots(figsize = (15,10))

sorted_periods = {'Dawn': 0, 'Morning': 1, 'Afternoon': 2, 'Evening': 3, 'Night': 4, 'Late Night': 5}
data['Day Period Value'] = data['index'].map(sorted_periods)
data = data.sort_values('Day Period Value').drop('Day Period Value',axis=1)

line = sns.lineplot(x= data['index'], y= data['Day Period'])
line.set_title('Crime occurrence over day periods')
line.set_ylabel('Count')
line.set_xlabel('Day Period')

In [ ]:
ct_period_cat = pd.crosstab(train_clean['Category'], train_clean['Day Period'])
ct_period_cat

In [ ]:
stacked = ct_period_cat.stack().reset_index().rename(columns={0:'value'})
sorted_periods = {'Dawn': 0, 'Morning': 1, 'Afternoon': 2, 'Evening': 3, 'Night': 4, 'Late Night': 5}
stacked['Day Period Value'] = stacked['Day Period'].map(sorted_periods)
stacked = stacked.sort_values('Day Period Value').drop('Day Period Value',axis=1)

fig, count = plt.subplots(figsize = (15,10))
line = sns.lineplot(x=stacked['Day Period'], y=stacked['value'], hue=stacked['Category'])
line.set_title('Crime occurrence over day periods')

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays= cal.holidays(start= train_clean['Dates'].min(), end= train_clean['Dates'].max())
train_clean['Holiday']= train_clean['Dates'].dt.date.astype('datetime64').isin(holidays)
train_clean['Holiday'].value_counts()

# pd.DataFrame(holidays)
# train_clean

In [ ]:
ct_holiday_cat = pd.crosstab(train_clean['Category'], train_clean['Holiday'])
ct_holiday_cat

In [ ]:
stacked = ct_holiday_cat.stack().reset_index().rename(columns={0:'value'})
stacked.loc[stacked['Holiday'] == False, 'value'] /= train_clean.loc[train_clean['Holiday'] == False, 'Holiday'].count()
stacked.loc[stacked['Holiday'] == True, 'value'] /= train_clean.loc[train_clean['Holiday'] == True, 'Holiday'].count()
stacked

In [ ]:
fig, count = plt.subplots(figsize = (15,10))
bar = sns.barplot(x=stacked['Category'], y=stacked['value'], hue=stacked['Holiday'])
bar.set_title('Proportions of crimes during regular days vs holidays')

In [ ]:
def business_hours(hours):
    return 8 <= hours <= 18

In [ ]:
train_clean['Business Hours'] = train_clean['HOUR'].map(business_hours)
train_clean['Business Hours'].value_counts()

In [ ]:
ct_business_hrs_cat = pd.crosstab(train_clean['Category'], train_clean['Business Hours'])
ct_business_hrs_cat

In [ ]:
stacked = ct_business_hrs_cat.stack().reset_index().rename(columns={0:'value'})
stacked.loc[stacked['Business Hours'] == False, 'value'] /= train_clean.loc[train_clean['Business Hours'] == False, 'Business Hours'].count()
stacked.loc[stacked['Business Hours'] == True, 'value'] /= train_clean.loc[train_clean['Business Hours'] == True, 'Business Hours'].count()
stacked

In [ ]:
fig, count = plt.subplots(figsize = (15,10))
bar = sns.barplot(x=stacked['Category'], y=stacked['value'], hue=stacked['Business Hours'])
bar.set_title('Proportions of crimes in business hours vs rest of the day')

In [ ]:
def year_quarter(month):
    if month >= 1 and month <= 3:
        return 'Q1'
    elif month > 3 and month <= 6:
        return 'Q2'
    elif month > 6 and month <= 9:
        return 'Q3'
    elif month > 9 and month <= 12:
        return 'Q4'

In [ ]:
train_clean['MONTH']
train_clean['Year Quarter'] = train_clean['MONTH'].map(year_quarter)
train_clean['Year Quarter'].value_counts()

In [ ]:
ct_yquarter_cat = pd.crosstab(train_clean['Category'], train_clean['Year Quarter'])
ct_yquarter_cat

In [ ]:
stacked = ct_yquarter_cat.stack().reset_index().rename(columns={0:'value'})
fig, line = plt.subplots(figsize = (15,10))

sorted_quarters = {'Q1': 0, 'Q2': 1, 'Q3': 2, 'Q4': 3}
stacked['Quarter Value'] = stacked['Year Quarter'].map(sorted_quarters)
stacked = stacked.sort_values('Quarter Value').drop('Quarter Value',axis=1)

line = sns.lineplot(x=stacked['Year Quarter'], y=stacked['value'], hue=stacked['Category'])
line.set_title('Crime occurrences per quarter year')

In [ ]:
def weekend(weekday):
    if weekday == "Saturday" or weekday == "Sunday":
        return True
    else:
        return False

In [ ]:
train_clean['Weekend'] = train_clean['DayOfWeek'].map(weekend)
train_clean['Weekend'].value_counts()

In [ ]:
ct_weekend_cat = pd.crosstab(train_clean['Category'], train_clean['Weekend'])
ct_weekend_cat

In [ ]:
stacked = ct_weekend_cat.stack().reset_index().rename(columns={0:'value'})
stacked.loc[stacked['Weekend'] == False, 'value'] /= train_clean.loc[train_clean['Weekend'] == False, 'Weekend'].count()
stacked.loc[stacked['Weekend'] == True, 'value'] /= train_clean.loc[train_clean['Weekend'] == True, 'Weekend'].count()
stacked

In [ ]:
fig, count = plt.subplots(figsize = (15,10))
bar = sns.barplot(x=stacked['Category'], y=stacked['value'], hue=stacked['Weekend'])
bar.set_title('Crime proportions on weekends vs normal days')

In [ ]:
train_clean

In [ ]:
def third_of_month(day):
    if day >= 1 and day < 10:
        return 'T1'
    elif day >= 10 and day < 20:
        return 'T2'
    elif day >= 20 and day <= 31:
        return 'T3'

In [ ]:
train_clean['Third_of_Month'] = train_clean['DAY'].map(third_of_month)
train_clean

In [ ]:
def type_of_street_extraction(Address):
    type_of_street= Address.split(' ')
    for index in range(len(type_of_street)):
        cell_value= type_of_street[index]
        if len(cell_value) == 2 and cell_value not in['of', 'OF', 'US', 'LA']:
            #ls_street_types.append(cell_value)
            return cell_value
            break

In [ ]:
train_clean['street_type'] = train_clean['Address'].map(type_of_street_extraction)

In [ ]:
train_clean['street_type'].unique()

In [ ]:
train_clean.info()

In [ ]:
train_clean.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_clean['Category_Labels'] = le.fit_transform(train_clean['Category'])
train_clean['DayOfWeek_Labels'] = le.fit_transform(train_clean['DayOfWeek'])
train_clean['PdDistrict_Labels'] = le.fit_transform(train_clean['PdDistrict'])
train_clean['Day Period'] = le.fit_transform(train_clean['Day Period'])
train_clean['Year Quarter'] = le.fit_transform(train_clean['Year Quarter'])
train_clean['street_type_Labels'] = le.fit_transform(train_clean['street_type'])
train_clean

In [ ]:
train_clean['Third_of_Month_Labels'] = le.fit_transform(train_clean['Third_of_Month'])

In [ ]:
train_clean = train_clean.drop(columns = ["Third_of_Month"])

In [ ]:
train_clean= train_clean.drop(columns=['Category', 'DayOfWeek','PdDistrict','Address', 'Day Period'
                                       , 'Year Quarter', 'street_type', 'Dates'])
train_clean

In [ ]:
train_clean= train_clean.drop(columns=['Resolution'])
train_clean = train_clean.drop(columns= ["Descript"])
train_clean

In [ ]:
train_clean['Holiday'].replace(False, 0, inplace=True)
train_clean["Holiday"].replace(True, 1, inplace = True)
train_clean["Holiday"] = train_clean["Holiday"].astype('uint8')

In [ ]:
train_clean['Business Hours'].replace(False, 0, inplace=True)
train_clean["Business Hours"].replace(True, 1, inplace = True)
train_clean["Business Hours"] = train_clean["Business Hours"].astype('uint8')

In [ ]:
train_clean['Weekend'].replace(False, 0, inplace=True)
train_clean["Weekend"].replace(True, 1, inplace = True)
train_clean["Weekend"] = train_clean["Weekend"].astype('uint8')

In [ ]:
train_clean

In [ ]:
train_clean['Category_Labels'].unique()

In [ ]:
# Set training data (drop labels) and training labels
X_train = train_clean.drop("Category_Labels", axis=1).copy()
Y_train = train_clean["Category_Labels"].copy()

In [ ]:
from sklearn.model_selection import train_test_split
# Use these for ML algorithms that can handle categorical data without OHE
mini_train_data, mini_dev_data, mini_train_labels, mini_dev_labels = train_test_split(X_train, 
                                                                                      Y_train,
                                                                                      stratify= Y_train,
                                                                                      test_size= 0.3,
                                                                                      random_state= 1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Random Forest Ensemble
# n_estimators: This is the number of trees you want to build before taking the maximum voting or averages of predictions.
# Higher number of trees give you better performance but makes your code slower.
random_forest = RandomForestClassifier(n_estimators= 550,
                                       max_depth= 31,
                                       max_features= 'sqrt',
                                       min_samples_leaf= 16,
                                       min_samples_split= 43, 
                                       random_state= 1,
                                       verbose= 1,
                                       n_jobs= 2)

In [ ]:
random_forest.fit(mini_train_data, mini_train_labels)

In [ ]:
pred_probs = random_forest.predict_proba(mini_dev_data)

In [ ]:
from sklearn.metrics import log_loss
rf_loss = log_loss(mini_dev_labels, pred_probs)

In [ ]:
print('Random Forest Validation Log Loss: ', rf_loss)